In [2]:
import numpy as np
import pandas as pd
pest_data = pd.read_csv("Data/BinaryPesticideData.csv", delimiter=",")
pest_data = pest_data.drop(['LOD'], axis=1)

X = pest_data[['ExactMolWt', 'HeavyAtomMolWt', 'NumValenceElectrons', 'HeavyAtomCount', 'NHOHCount', 
                        'NOCount', 'NumAliphaticCarbocycles', 'NumAliphaticHeterocycles', 'NumAliphaticRings', 
                        'NumAromaticCarbocycles', 'NumAromaticHeterocycles', 'NumAromaticRings', 'NumHAcceptors', 
                        'NumHDonors', 'NumHeteroatoms', 'NumRotatableBonds', 'NumSaturatedCarbocycles', 
                        'NumSaturatedHeterocycles', 'NumSaturatedRings', 'RingCount', 'Halogens', "MolecularWeight", 
                        "XLogP", "ExactMass", "TPSA", "Complexity", "Charge", "HBondDonorCount", 
                        "HBondAcceptorCount", "RotatableBondCount", "HeavyAtomCount", "IsotopeAtomCount", "AtomStereoCount", 
                        "DefinedAtomStereoCount", "UndefinedAtomStereoCount", "BondStereoCount", "DefinedBondStereoCount", 
                        "UndefinedBondStereoCount", "CovalentUnitCount","Volume3D", "XStericQuadrupole3D", "YStericQuadrupole3D", 
                        "ZStericQuadrupole3D", "FeatureCount3D", "FeatureAcceptorCount3D", "FeatureDonorCount3D", "FeatureAnionCount3D", 
                        "FeatureCationCount3D", "FeatureRingCount3D", "FeatureHydrophobeCount3D", "ConformerModelRMSD3D", 
                        "EffectiveRotorCount3D", "ConformerCount3D"]].values


y = pest_data[['Class']].values
y_array = pest_data[['Class']].to_numpy()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle=True) #random_state = 3



In [85]:
from hpsklearn import HyperoptEstimator #any_sparse_classifier
from hyperopt import tpe
from hpsklearn import k_neighbors_classifier

model_KNN = HyperoptEstimator(classifier=k_neighbors_classifier('myKNN'), algo=tpe.suggest, max_evals=300, trial_timeout=500)
# perform the search
model_KNN.fit(X_train, y_train.ravel())

 97%|█████████▋| 28/29 [00:23<?, ?trial/s, best loss=?]


In [4]:
# show the results
print(model_KNN.score(X_test, y_test))
print(model_KNN.best_model())

0.5208333333333334
{'learner': DecisionTreeClassifier(criterion='entropy', max_features=0.025940389422746013,
                       random_state=2), 'preprocs': (Normalizer(norm='max'),), 'ex_preprocs': ()}


In [5]:
model_KNN.predict(X_test)

array([2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2,
       1, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2,
       2, 2, 1, 1], dtype=int64)

In [80]:
import flask
import pickle
from urllib.request import urlopen
from urllib.parse import quote
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import Lipinski
from rdkit.Chem import Fragments
import pubchempy as pcp


def genDesc(name):
    sm = CIRconvert(name)
    mol = Chem.MolFromSmiles(sm)
    desc = []

    #RDKit descriptors
    desc.append(Descriptors.ExactMolWt(mol))
    desc.append(Descriptors.HeavyAtomMolWt(mol))
    desc.append(Descriptors.NumValenceElectrons(mol))
    desc.append(Lipinski.HeavyAtomCount(mol))
    desc.append(Lipinski.NHOHCount(mol))
    desc.append(Lipinski.NOCount(mol))
    desc.append(Lipinski.NumAliphaticCarbocycles(mol))
    desc.append(Lipinski.NumAliphaticHeterocycles(mol))
    desc.append(Lipinski.NumAliphaticRings(mol))
    desc.append(Lipinski.NumAromaticCarbocycles(mol))
    desc.append(Lipinski.NumAromaticHeterocycles(mol))
    desc.append(Lipinski.NumAromaticRings(mol))
    desc.append(Lipinski.NumHAcceptors(mol))
    desc.append(Lipinski.NumHDonors(mol))
    desc.append(Lipinski.NumHeteroatoms(mol))
    desc.append(Lipinski.NumRotatableBonds(mol))
    desc.append(Lipinski.NumSaturatedCarbocycles(mol))
    desc.append(Lipinski.NumSaturatedHeterocycles(mol))
    desc.append(Lipinski.NumSaturatedRings(mol))
    desc.append(Lipinski.RingCount(mol))
    desc.append(Fragments.fr_halogen(mol))
    #return desc.values

    descDF = pd.DataFrame(np.reshape(desc, (1, -1)), columns = ['ExactMolWt', 'HeavyAtomMolWt', 'NumValenceElectrons', 'HeavyAtomCount', 'NHOHCount', 
                        'NOCount', 'NumAliphaticCarbocycles', 'NumAliphaticHeterocycles', 'NumAliphaticRings', 
                        'NumAromaticCarbocycles', 'NumAromaticHeterocycles', 'NumAromaticRings', 'NumHAcceptors', 
                        'NumHDonors', 'NumHeteroatoms', 'NumRotatableBonds', 'NumSaturatedCarbocycles', 
                        'NumSaturatedHeterocycles', 'NumSaturatedRings', 'RingCount', 'Halogens'], index = [pcp.get_compounds(sm, "smiles")[0].cid])

    #PubChem descriptors
    features_2D = ["MolecularWeight", "XLogP", "ExactMass", "TPSA",
                   "Complexity", "Charge", "HBondDonorCount", "HBondAcceptorCount", "RotatableBondCount",
               "HeavyAtomCount", "IsotopeAtomCount", "AtomStereoCount", "DefinedAtomStereoCount", "UndefinedAtomStereoCount",
               "BondStereoCount", "DefinedBondStereoCount", "UndefinedBondStereoCount", "CovalentUnitCount"]
    desc_2D = pcp.get_properties(features_2D, sm, "smiles", as_dataframe=True)

    features_3D = ["Volume3D", "XStericQuadrupole3D", "YStericQuadrupole3D", "ZStericQuadrupole3D", "FeatureCount3D",
               "FeatureAcceptorCount3D", "FeatureDonorCount3D", "FeatureAnionCount3D", "FeatureCationCount3D",
               "FeatureRingCount3D", "FeatureHydrophobeCount3D", "ConformerModelRMSD3D", "EffectiveRotorCount3D",
               "ConformerCount3D"]
    desc_3D = pcp.get_properties(features_3D, sm, "smiles", as_dataframe=True)
    
    overall = pd.concat([descDF, desc_2D, desc_3D], axis=1)
    return overall

def CIRconvert(ids):
    try:
        url = 'http://cactus.nci.nih.gov/chemical/structure/' + quote(ids) + '/smiles'
        ans = urlopen(url).read().decode('utf8')
        return ans
    except:
        return 'Did not work'



In [83]:
input_variables = genDesc("Carfentrazone ethyl")
model_DT.predict(input_variables)[0]

2

In [ ]:
# save the best model
import pickle
with open('model_DT.pkl', 'wb') as f:
    pickle.dump(model_DT, f)
